# Get distance along shortest path  between two locations within Garching

In [130]:
import psycopg2
import networkx as nx
from itertools import tee
import time 
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

import graph_wrapper as gw
%load_ext autoreload
%autoreload 2

def cleanMap(center, boundary): 
    # Mark transformer
    tMarker =  Marker(location=center)
    # Mark polygon
    tPoly = Polygon(locations=boundary, weight=3,
                color='#003d99', opacity=0.8, fill_opacity=0.0,
                fill_color='#ccffcc') 
    tMap = Map(default_tiles=TileLayer(opacity=1.0),center=center ,zoom=13)
    tMap.add_layer(tMarker)
    tMap.add_layer(tPoly)
    return tMap

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Connect to remote db 

In [133]:
hostaddr = raw_input('Enter server address:')
db = raw_input('Enter database to connect:')
user = raw_input('Enter username:')
password = raw_input('Enter password:')
conn = gw.connect_to_psql(dbname=db, user=user, password=password, hostaddr=hostaddr, remote=True)

## Split roads within Garching and generate connected subgraphs

In [122]:
# Locate to Garching
GARCHING = -30971
garchingCenter, garchingPoly = gw.get_area(conn, reg_osm_id = GARCHING)

"""
# Execute following code to update the result of split_roads stored in database
# Select roads of specified types
road_type=['motorway', 'trunk', 'primary', 'secondary', 'unclassified','tertiary', 
           'residential', 'service', 'motorway_link', 'trunk_link', 
           'primary_link', 'secondary_link', 'tertiary_link', 
           'living_street', 'pedestrian', 'road', 'footway']
roads = gw.get_roads(conn, reg_osm_id=-30971, road_type=road_type)

# Split roads of specified types
t1 = time.time()
split_roads_coords = gw.split_by_difference(roads)
print 'Totoal spliting time : %s' % (time.time() - t1) 
print 'Original roads: %d Split roads: %d ' % (len(roads), len(split_roads))

# Store split result for convenience
gw.store_split_roads(conn, split_roads_coords):
"""
# Retrieve split roads stored in database
split_roads = gw.fetch_split_roads(conn)
# Generate subgraphs by connecting split roads
subgraphs = gw.generate_connected_subgarphs(split_roads)

## Check split roads

In [128]:
tMap = cleanMap(garchingCenter, garchingPoly)
for rd in split_roads:
    cs = Circle(location=rd[0], weight=2, opacity = 0.9, color = 'blue', radius = 3) 
    ce = Circle(location=rd[len(rd)-1], weight=2, opacity = 0.9, color = 'green', radius = 3) 
    tMap.add_layer(cs)
    tMap.add_layer(ce)
tMap

## Test and visualisation

In [129]:
# Test sample
source = (48.2515727076087, 11.6554101829255)#(48.2413, 11.6579)
target = (48.2533079903546, 11.6556654277995)#(48.25150, 11.64837)

path, length, graph = gw.find_shortest_path(source, target, subgraphs)
print 'Total length %f' % length

# Show on map
tMap = cleanMap(garchingCenter, garchingPoly)
srcMarker = Circle(location=source, weight=2, opacity = 0.9, color = 'blue', radius = 3) 
tarMarker = Circle(location=target, weight=2, opacity = 0.9, color = 'green', radius = 3) 
tMap.add_layer(srcMarker)
tMap.add_layer(tarMarker)

sid, eid = tee(path) # Initial index of edge start and end node
next(eid) # Move end node next to start
for n1, n2 in zip(sid, eid):
    pl = Polyline(locations=(n1,n2), weight=2, color='red', opacity=0.7) 
    tMap.add_layer(pl)
tMap

## Check roads connection

In [132]:
# Select roads of specified types
road_type=['motorway', 'trunk', 'primary', 'secondary', 'unclassified','tertiary', 
           'residential', 'service', 'motorway_link', 'trunk_link', 
           'primary_link', 'secondary_link', 'tertiary_link', 
           'living_street', 'pedestrian', 'road', 'footway']
roads = gw.get_roads(conn, reg_osm_id=-30971, road_type=road_type)